# **Diplomado de Machine Learning con Python**
## Tarea 04 – Sesión 06

## **Codificación de variables**

📊 En el contexto de machine learning, muchas técnicas requieren que todas las variables de entrada sean numéricas. Sin embargo, los conjuntos de datos reales suelen contener variables categóricas, como nombres ciudades, estado civil u otras. Para que estas variables puedan ser utilizadas por algoritmos, es necesario transformarlas mediante técnicas de codificación. Dos enfoques comunes son One-Hot Encoding y Ordinal Encoding.

La elección del método depende del tipo de variable y del modelo que se va a utilizar. Una codificación adecuada mejora la capacidad del modelo para aprender patrones relevantes y evita errores por interpretaciones numéricas incorrectas.

Para esta tarea se utilizará el **Palmer Penguins Extended Dataset**. 

![Palmer Penguins Illustration](https://allisonhorst.github.io/palmerpenguins/logo.png)


## Importación de librerías y lectura de archivo csv

In [19]:
# Importar librerías
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# Leer el CSV
df = pd.read_csv("palmerpenguins_extended.csv")
print(f'Dimensiones del dataset: Filas: {df.shape[0]}, Columnas: {df.shape[1]} \n')
df.head(5)

Dimensiones del dataset: Filas: 3430, Columnas: 11 



,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,diet,life_stage,health_metrics,year
0,Adelie,Biscoe,53.4,17.8,219.0,5687.0,female,fish,adult,overweight,2021
1,Adelie,Biscoe,49.3,18.1,245.0,6811.0,female,fish,adult,overweight,2021
2,Adelie,Biscoe,55.7,16.6,226.0,5388.0,female,fish,adult,overweight,2021
3,Adelie,Biscoe,38.0,15.6,221.0,6262.0,female,fish,adult,overweight,2021
4,Adelie,Biscoe,60.7,17.9,177.0,4811.0,female,fish,juvenile,overweight,2021


## Revisión inicial de los datos

In [20]:
d_type = df.dtypes # Tipo de dato
n_non_null = df.count() # Numero de valores no nulos
n_unique = df.nunique() # Numero de valores unicos
n_null = df.isnull().sum() # Numero de valors nulos
ratio_null = df.isnull().sum()/df.shape[0] # Porcentaje de valores nulos

pd.DataFrame(
    {"d_type": d_type,
     "n_non_null": n_non_null,
     "n_unique": n_unique,
     "n_null": n_null,
     "ratio_null": ratio_null}
)

,d_type,n_non_null,n_unique,n_null,ratio_null
species,object,3430,3,0,0.0
island,object,3430,3,0,0.0
bill_length_mm,float64,3430,572,0,0.0
bill_depth_mm,float64,3430,169,0,0.0
flipper_length_mm,float64,3430,159,0,0.0
body_mass_g,float64,3430,2428,0,0.0
sex,object,3430,2,0,0.0
diet,object,3430,4,0,0.0
life_stage,object,3430,3,0,0.0
health_metrics,object,3430,3,0,0.0


## Definir el tipo de codficación para cada columna

### 🧩 One-Hot Encoding

One-Hot Encoding transforma variables categóricas en columnas binarias, creando una nueva columna para cada categoría posible. Cada fila tiene un valor de 1 en la columna correspondiente a su categoría y 0 en las demás.

Esta técnica es ideal para variables sin orden lógico, como colores, géneros o tipos de producto, ya que evita que el modelo interprete relaciones numéricas inexistentes entre las categorías. Aunque puede generar muchas columnas si hay muchas categorías, es una forma segura y efectiva de representar información categórica en modelos matemáticos.

### 🎯 Ordinal Encoding

Ordinal Encoding asigna un número entero a cada categoría, respetando su orden natural. Por ejemplo, niveles como "bajo", "medio" y "alto" pueden codificarse como 0, 1 y 2. Esta técnica es útil cuando las categorías tienen una jerarquía implícita, ya que permite al modelo capturar esa relación.
Sin embargo, si se aplica a variables sin orden real, puede inducir errores al sugerir una progresión que no existe. Por eso, es importante usarla solo cuando el orden entre categorías tiene sentido en el contexto del análisis.


In [22]:
# Columnas
cols_onehot  = ['species', 'island', 'sex', 'diet']   # NOMINALES → One-Hot
cols_ordinal = ['life_stage','health_metrics']      # ORDINAS → Ordinal

# Categorías ordenadas para las ordinales (mismo orden que en cols_ordinal)
categorias_ordinales = [
    ["chick", "juvenile", "adult"],  # turno
    ["underweight", "healthy", "overweight"]  # nivel_prioridad
]

print(f'Columnas para codifiación One-Hot:\n {cols_onehot}\n')
print(f'Columnas para codificación ordinal:\n {cols_ordinal}\n')

Columnas para codifiación One-Hot:
 ['species', 'island', 'sex', 'diet']

Columnas para codificación ordinal:
 ['life_stage', 'health_metrics']



## Aplicar transformaciones a las columnas con `ColumnTransformer`

In [23]:
'''
Explicación del ColumnTransformer usado

1) transformers = [ ... ]
   Cada tupla es (nombre_bloque, transformador, columnas_objetivo)

   a) ("onehot", OneHotEncoder(...), cols_onehot)
      - Aplica One-Hot Encoding a columnas nominales.
      - sparse_output=False  → salida densa (más cómodo para DataFrame).
      - drop=None            → no se descarta ninguna categoría.
      - handle_unknown="ignore" → si aparece una categoría NO vista en fit,
        no lanza error; la fila queda con ceros en todas las dummies de esa variable.

   b) ("ordinal", OrdinalEncoder(...), cols_ordinal)
      - Aplica codificación ordinal respetando un ORDEN explícito por columna.
      - categories=categorias_ordinales → listas con el orden de cada variable.
      - handle_unknown="use_encoded_value", unknown_value=-1  → cualquier
        categoría NO vista en fit se codifica como -1 (fuera del rango normal 0..k-1).
        Esto permite detectar fácilmente valores "desconocidos" en datos nuevos.

Resumen:
- One-Hot para nominales (sin orden), tolerante a categorías nuevas (se codifican como todo ceros).
- Ordinal para variables con orden natural; categorías nuevas se marcan con -1.
- El resto de columnas se descartan (remainder="drop").
'''

preprocesador = ColumnTransformer(
    transformers=[
        ("onehot",
         OneHotEncoder(sparse_output=False, drop=None, handle_unknown="ignore"),
         cols_onehot),
        ("ordinal",
         OrdinalEncoder(categories=categorias_ordinales,
                        handle_unknown="use_encoded_value", unknown_value=-1),
         cols_ordinal),
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

preprocesador.fit(df)

X_encoded = preprocesador.transform(df)


## Reconstruir DataFrame con las variables transformadas


In [24]:
# Reconstruir DataFrame con nombres de columnas

# Obtiner los nombres de las columnas resultantes del preprocesador
cols_out = preprocesador.get_feature_names_out()

# Convertir la matriz NumPy codificada (X_encoded) en un DataFrame de pandas
df_encoded = pd.DataFrame(X_encoded, columns=cols_out, index=df.index)

# Muestra el DataFrame codificado, con variables categóricas transformadas en formato numérico
df_encoded


,species_Adelie,species_Chinstrap,species_Gentoo,island_Biscoe,island_Dream,island_Torgensen,sex_female,sex_male,diet_fish,diet_krill,diet_parental,diet_squid,life_stage,health_metrics
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3425,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0
3426,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0
3427,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,2.0
3428,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0


In [2]:
# Exportar DataFrame obtenido a CSV

df_encoded.to_csv("data_encoded.csv")

NameError: name 'df_encoded' is not defined